In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [18]:
dist_df = pd.read_csv('data/상권-추정매출_업종재분류_시계열.csv', encoding='cp949')

In [19]:
dist_df.head(2)

,상권_코드,업종_재분류,상권_구분_코드,상권_구분_코드_명,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,시군구_명,...,2020-Q4,2021-Q1,2021-Q2,2021-Q3,2021-Q4,2022-Q1,2022-Q2,2022-Q3,2022-Q4,2023-Q1
0,3110008,개인교통,A,골목상권,배화여자대학교(박노수미술관),197093,453418,11110,11110515,종로구,...,120234965.0,142797537.0,164324758.0,207912466.0,205394252.0,97594430.0,198299594.0,184570261.0,187961663.0,163676103.0
1,3110008,교육,A,골목상권,배화여자대학교(박노수미술관),197093,453418,11110,11110515,종로구,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10919665.0


## 업종별 매출비율 계산

2019-Q4 만 사용

In [20]:
dist_df = dist_df[['상권_코드', '업종_재분류', '2019-Q4']]

In [21]:
SECTORS = dist_df['업종_재분류'].unique()
SECTORS

array(['개인교통', '교육', '기타', '리빙', '뷰티', '숙박', '온라인판매', '외식업', '의료기기',
       '패션소품', '편의점', '홈쿡', '병원', '스포츠', '실내다중', '인테리어', '부동산'],
      dtype=object)

In [22]:
dist_df.isna().sum()

상권_코드         0
업종_재분류        0
2019-Q4    3083
dtype: int64

In [23]:
dist_df.fillna(0, inplace=True)

In [24]:
dist_tot = dist_df.groupby('상권_코드').sum().reset_index()

C:\Users\msl06\AppData\Local\Temp\ipykernel_18448\2710576731.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dist_tot = dist_df.groupby('상권_코드').sum().reset_index()


In [25]:
dist_tot.head(2)

,상권_코드,2019-Q4
0,3110001,1.629469e+09
1,3110002,1.063425e+10


In [26]:
q4_sales_dict = dict(zip(dist_tot['상권_코드'], dist_tot['2019-Q4']))

In [27]:
dist_df['q4_sales_total'] = dist_df['상권_코드'].map(q4_sales_dict)

In [28]:
dist_df['ratio'] = dist_df['2019-Q4'] / dist_df['q4_sales_total']

In [29]:
dist_df.set_index(['상권_코드', '업종_재분류'], inplace=True)

In [36]:
sales_df = pd.read_csv('data/resilience_index.csv', encoding='cp949')

In [37]:
sales_df.head(2)

,Unnamed: 0,상권_코드,2019-Q4,2020-Q1,2020-Q2,2020-Q3,2020-Q4,2021-Q1,2021-Q2,2021-Q3,...,y_pred_6_dtw,Covid_local_min,t_recovery,d_resistance,d_recovery,d_stability,drop,rebound,res_ratio,resilience
0,0,3110001,0.0,-0.073992,0.037308,-0.020321,0.003290,-0.057947,0.004617,-0.027532,...,0,2020-Q1,2020-Q2,1,1,10,0.156650,0.292112,0.954901,0.741137
1,1,3110002,0.0,-0.060906,-0.108002,-0.115149,-0.106439,-0.123700,-0.127102,-0.124724,...,2,2021-Q2,2022-Q4,6,6,0,0.253727,0.021693,-0.604138,-0.558667


In [38]:
sales_df.drop('Unnamed: 0', axis=1, inplace=True)

In [39]:
for sector in SECTORS:
    sales_df[f'{sector}_비율'] = sales_df['상권_코드'].apply(
        lambda x: dist_df.loc[(x, sector), 'ratio'] 
        if (x, sector) in dist_df.index else 0)

In [40]:
sales_df.head(2)

,상권_코드,2019-Q4,2020-Q1,2020-Q2,2020-Q3,2020-Q4,2021-Q1,2021-Q2,2021-Q3,2021-Q4,...,외식업_비율,의료기기_비율,패션소품_비율,편의점_비율,홈쿡_비율,병원_비율,스포츠_비율,실내다중_비율,인테리어_비율,부동산_비율
0,3110001,0.0,-0.073992,0.037308,-0.020321,0.003290,-0.057947,0.004617,-0.027532,0.019208,...,0.527840,0.000000,0.0,0.147629,0.059447,0.000000,0.017604,0.023376,0.00000,0.0
1,3110002,0.0,-0.060906,-0.108002,-0.115149,-0.106439,-0.123700,-0.127102,-0.124724,-0.062470,...,0.132358,0.331851,0.0,0.176979,0.018628,0.016632,0.000766,0.012993,0.08734,0.0


In [41]:
sales_df.to_csv('data/dist_stats.csv', index=False, encoding='cp949')